In [ ]:
"""

Title: "Neural Barrier Certificates for Monotone Systems"
Code for: Learing an embedded neural barrier certificate for a 2-D linear example.
Author: Amirreza Alavi
Contact: seyedamirreza.alavi@colorado.edu
Affiliation: University of Colorado Boulder
Date: 03/2025

"""

In [ ]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from numpy import linalg as LA
from scipy import stats
from functools import partial
%matplotlib inline
import numpy as np
from tensorflow.keras.layers import Input, Dense, Concatenate, Add
from tensorflow.keras import Model
from tensorflow.keras import regularizers
import time 
import matplotlib.patches as mpatches

In [ ]:
def cart(x,y):
    temp = np.zeros([x.shape[0]*y.shape[0],x.shape[1]+y.shape[1]])
    #num1 = y.shape[1]
    j=0
    num2 = x.shape[0]
    num1 = x.shape[1]
    for i in range(temp.shape[0]):
        if i!=0 and i%num2 == 0:
            j+=1
        temp[i,0:num1] = x[i%num2]
        temp[i,num1::] = y[j]
    return temp

z0bar = np.array([[-1 , 2]])
z0und = np.array([[-2 , 1]])
z00 = np.concatenate([z0bar, -z0und], axis=1)

smpnz10=5
smpnz20=5
z10 = np.linspace(-2,-1,smpnz10).reshape(smpnz10,1)
z20 = np.linspace(1,2,smpnz20).reshape(smpnz20,1)
z0 = cart (z10,z20)

zubar = np.array([[2 ,  1]])
zuund = np.array([[1 ,  -1]])
zuu = np.concatenate([zuund, -zubar], axis=1)

smpnz11u=5
smpnz21u=9
z11u = np.linspace(1,2,smpnz11u).reshape(smpnz11u,1)
z21u = np.linspace(-1,1,smpnz21u).reshape(smpnz21u,1)
zu = cart (z11u,z21u)

smpnz1=5
smpnz2=5
z1 = np.linspace(-2, 2, smpnz1).reshape(smpnz1,1)
z2 = np.linspace(-2, 2, smpnz2).reshape(smpnz2,1)
z = cart (z1,z2)

pairs = np.zeros((((smpnz1-1)*(smpnz2)),2,2))
excluded_numbers = {i for i in range(smpnz1 - 1, (smpnz1 * smpnz2) - (smpnz1+1) , smpnz1)}
for i in range((smpnz1 * smpnz2) - (smpnz1+1)):
    if i not in excluded_numbers:
        pairs [i,:,:] = np.array([z[i + smpnz1 + 1], z[i]])

zz = pairs[~np.all(pairs == 0, axis=(1, 2))]

unsafe_mask = (
    ((zz[:, 0, 0] >= 1) & (zz[:, 0, 0] <= 2) &   
     (zz[:, 0, 1] >= -1) & (zz[:, 0, 1] <= 1)) &   
    ((zz[:, 1, 0] >= 1) & (zz[:, 1, 0] <= 2) &   
     (zz[:, 1, 1] >= -1) & (zz[:, 1, 1] <= 1))     
)

zz_filtered = zz[~unsafe_mask]

zbar = zz_filtered [:,0,:]
zund = zz_filtered [:,1,:]
zz = np.concatenate([zund, -zbar], axis=1)

h=0.1

z1bar = tf.reshape(zbar[:,0].astype(np.float32),[zbar.shape[0],1])
z2bar = tf.reshape(zbar[:,1].astype(np.float32),[zbar.shape[0],1])

z1und = tf.reshape(zund[:,0].astype(np.float32),[zund.shape[0],1])
z2und = tf.reshape(zund[:,1].astype(np.float32),[zund.shape[0],1])

z1barn = (1-2*h)*z1bar + (h)*z2bar - h
z2barn = (h)*z1bar + (1-2*h)*z2bar + h

z1undn = (1-2*h)*z1und + (h)*z2und - h
z2undn = (h)*z1und + (1-2*h)*z2und + h

fzbar = tf.reshape(tf.stack([z1barn,z2barn],axis=1),zbar.shape)
fzund = tf.reshape(tf.stack([z1undn,z2undn],axis=1),zund.shape)

fzz = np.concatenate([fzbar, -fzund], axis=1)

In [ ]:
# B

modelB = tf.keras.Sequential()
modelB.add(layers.Dense(40, use_bias=True, activation = 'tanh' ,input_shape=(4,),kernel_constraint=tf.keras.constraints.NonNeg()))
modelB.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelB.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelB.add(layers.Dense(60, use_bias=True, activation = 'tanh' ,kernel_constraint=tf.keras.constraints.NonNeg()))
modelB.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelB.add(layers.Dense(1,use_bias=True, activation = 'linear',kernel_constraint=tf.keras.constraints.NonNeg()))

opt1 = tf.keras.optimizers.Adam(1e-2)
opt2 = tf.keras.optimizers.Adam(1e-2)

In [ ]:
epochs = 300000

t1 = time.perf_counter()
eta=0.1
etaa=0.0000001

for i in range(epochs):
    with tf.GradientTape() as tape0, tf.GradientTape() as tape1, tf.GradientTape() as tape2, tf.GradientTape() as tape3:
        
        
        B_z0 = tf.reshape(modelB(z00, training=True), [z00.shape[0], 1])
        B_zu = tf.reshape(modelB(zuu, training=True), [zuu.shape[0], 1])
        B_z = tf.reshape(modelB(zz, training=True), [zz.shape[0], 1])
        B_fz = tf.reshape(modelB(fzz, training=True), [fzz.shape[0], 1])

   
        loss_h3 = tf.reduce_sum(tf.nn.relu(tf.subtract(eta,eta)))
        loss_h4 = tf.reduce_sum(tf.nn.relu(tf.subtract(eta,eta)))



        loss_h3 += tf.reduce_sum(tf.nn.relu(tf.subtract(B_z0,-etaa))) 
        loss_h3 += tf.reduce_sum(tf.nn.relu(0-(B_zu)))
        loss_h3 += tf.reduce_sum(tf.math.sign(tf.nn.relu(-(B_z)))* tf.nn.relu(B_fz))

        
        loss_h4 += tf.reduce_sum(tf.square(tf.nn.relu(B_z0 + 5))) 
        loss_h4 += tf.reduce_sum(tf.square(tf.nn.relu(1-B_zu)))
        loss_h4 += tf.reduce_sum(tf.math.sign(tf.nn.relu(-(B_z)))* tf.square(tf.nn.relu(B_fz)))

        if tf.reduce_sum(loss_h3) == 0 and i > 1:
            print(f"Training terminated at epoch {i} as loss reached exactly zero.")
            break
        
        grad_B = tape0.gradient(loss_h4, modelB.trainable_variables)
        
        clipgrad_B = [tf.clip_by_norm(g, 2) for g in grad_B]
        opt1.apply_gradients(zip(clipgrad_B, modelB.trainable_variables))


        if i % 20 == 0:
            print(f"Epoch {i}: loss_h3 = {loss_h3.numpy()}, loss_h4 = {loss_h4.numpy()}")
            
    
t2 = time.perf_counter()
tot = t2 - t1
print(f'Total runtime: {tot}s')


In [ ]:
x = np.linspace(-2, 2, 500)  
y = np.linspace(-2, 2, 500)
z1, z2 = np.meshgrid(x, y)
zp = np.stack([z1, z2], axis=-1).reshape(-1, 2) 
z = np.concatenate([zp, -zp], axis=1)

modelB_output = modelB.predict(z)  

condition = modelB_output
condition = condition.reshape(500, 500) 


plt.figure(figsize=(5, 4))
plt.scatter(z0[:, 0], z0[:, 1], color='blue', label='Initial Set ($X_0$)', alpha=1)
plt.scatter(zu[:, 0], zu[:, 1], color='red', label='Unsafe Set ($X_u$)', alpha=1)
plt.contourf(z1, z2, condition, levels=[-np.inf, 0], colors=['lightgray'], alpha=0.4)
plt.contour(z1, z2, condition, levels=[0], colors=['black'])
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


for i in range(zbar.shape[0]):
    width = zbar[i, 0] - zund[i, 0]
    height = zbar[i, 1] - zund[i, 1]
    rect = plt.Rectangle(zund[i], width, height, linewidth=1, edgecolor='green', facecolor='none')
    plt.gca().add_patch(rect)

gray_patch = mpatches.Patch(color='lightgray', label=r'N(x) $\leq$ 0')
plt.legend(handles=[gray_patch] + plt.gca().get_legend_handles_labels()[0], loc='lower left')


plt.rcParams.update({
    "font.weight": "normal",         
    "axes.labelweight": "normal",    
    "axes.titleweight": "normal",    
    "axes.labelsize": 12,         
    "xtick.labelsize": 12,         
    "ytick.labelsize": 12          
})

plt.rcParams["legend.fontsize"] = 11
plt.show()
